### Regressão Linear Analítica

In [32]:
import pandas as pd
import numpy as np
import time, psutil, os
import itertools
import torch

rows = []
cols = df_encoded.columns.tolist()

for x_col, y_col in itertools.permutations(cols, 2):  # assimétrico: x→y e y→x
    start_time = time.time()
    process = psutil.Process(os.getpid())

    # Converte para tensores
    x = torch.tensor(df_encoded[x_col].values, dtype=torch.float32)
    y = torch.tensor(df_encoded[y_col].values, dtype=torch.float32)

    # Monta matriz de projeto: [1, x]
    X = torch.stack([torch.ones_like(x), x], dim=1)

    # Regressão linear fechada: w = (X⁺ y)
    w = (torch.linalg.pinv(X) @ y).view(-1)  # [w0, w1]

    # Predição
    yhat = X @ w

    # Métricas
    # ===== Estatísticas clássicas =====
    N = len(y)
    ss_res = torch.sum((y - yhat)**2)          # SSE
    ss_tot = torch.sum((y - torch.mean(y))**2) # SST
    ss_reg = torch.sum((yhat - torch.mean(y))**2) # SSR
    r2  = 1.0 - (ss_res / (ss_tot + 1e-12))
    mse = torch.mean((y - yhat)**2)

    # ===== Tempo e memória =====
    wall_time_s = time.time() - start_time
    peak_mem_mb = process.memory_info().rss / 1024**2

    # ===== Resultados =====
    rows.append({
        "kind": "analytical",
        "x": x_col, "y": y_col,
        "w0": float(w[0]), "w1": float(w[1]),
        "R2": float(r2), "MSE": float(mse),
        "SSE": float(ss_res), "SSR": float(ss_reg), "SST": float(ss_tot),
        "N": N,
        "wall_s": wall_time_s,
        "peak_mb": peak_mem_mb,
        "iters": None,
        "grad_norm": None,
        "gap_w": None,
        "cond(X)": float(torch.linalg.cond(X))
    })

# Resultado final
result = pd.DataFrame(rows).sort_values(by=["R2", "MSE"], ascending=[False, True]).reset_index(drop=True)

if hasattr(args, "top_k") and args.top_k is not None:
    result = result.head(args.top_k)

display(result)



NameError: name 'df_encoded' is not defined

### PLOT

In [ ]:
@torch.no_grad()
def plot_pair(df_num: pd.DataFrame, x_col: str, y_col: str, title_suffix: str = ""):
    # tensores
    x = torch.tensor(df_num[x_col].values, dtype=torch.float32)
    y = torch.tensor(df_num[y_col].values, dtype=torch.float32)
    X = design_matrix_1d(x)
    w = closed_form_line(X, y)
    yhat = (X @ w)

    # métricas
    m = metrics(y, yhat)

    # para linhas suaves, ordenar por x
    order = torch.argsort(x)
    x_ord = x[order].numpy()
    yhat_ord = yhat[order].numpy()

    # média de Y
    y_bar = float(torch.mean(y))

    # --- PLOT ---
    plt.figure(figsize=(7.2, 5.0))

    # 1) Dispersão dos dados
    plt.scatter(x.numpy(), y.numpy(), label="Dados (Y vs X)", alpha=0.8)

    # 2) Reta de regressão (ŷ = w0 + w1*x) (traçada sobre x ordenado)
    plt.plot(x_ord, yhat_ord, color="red", linewidth=2.0, label="Reta de regressão (ŷ)")

    # 3) Linha horizontal: média de Y
    plt.axhline(y=y_bar, linestyle="--", color="green", linewidth=1.5, label="Média de Y (ȳ)")

    # 4) Resíduos
    xx = x.numpy()
    yy = y.numpy()
    yyhat = yhat.numpy()
    for xi, yi, ypi in zip(xx, yy, yyhat):
        # segmento vertical (do previsto ŷ ao observado y)
        plt.vlines(x=xi, ymin=min(yi, ypi), ymax=max(yi, ypi), linewidth=0.8, color="gray", alpha=0.5)

    # 5) Título e legendas
    ttl = f"{y_col} ~ {x_col}"
    if title_suffix:
        ttl += f" — {title_suffix}"
    plt.title(ttl)
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.legend(loc="best")
    plt.grid(alpha=0.25)

    # 6) Anotações das métricas no canto
    txt = (f"R² = {m['R2']:.4f}\n"
           f"MSE = {m['MSE']:.4g}\n"
           f"SST = {m['SST']:.4g}\n"
           f"SSR = {m['SSR']:.4g}\n"
           f"SSE = {m['SSE']:.4g}")
    plt.gcf().text(1.5, 0.5, txt, ha="right", va="bottom", fontsize=9,
                   bbox=dict(boxstyle="round", alpha=0.08, edgecolor="none"))

    plt.tight_layout()
    plt.show()

# ================== Plot Manager ==================
if args.plot:
    for _, row in result.iterrows():
        x_col = row["x"]
        y_col = row["y"]

        plot_pair(df_num, x_col, y_col, title_suffix=f"R²={row['R2']:.3f}")
else:
    print("[INFO] Para plotar execute usando a opção (args.plot = True).")


[INFO] Para plotar execute usando a opção (args.plot = True).
